In [2]:
import numpy as numpy
import pandas as pd
import os
import regex as re
#from categories import rules
import sys
print(sys.path)

['c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\python311.zip', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\DLLs', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\Lib', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health', '', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\Lib\\site-packages', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\Lib\\site-packages\\win32', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\jhuang\\AppData\\Local\\anaconda3\\envs\\health\\Lib\\site-packages\\setuptools\\_vendor']


In [4]:
rules = {
    'Lifestyle': {
        'mode': 'map',
        'factors': {
            'Work': {'None': 0, 'Little': 1, 'Moderate': 2, 'A lot': 3},
            'Stress': {'None': 0, 'Little': 1, 'Moderate': 2, 'A lot': 3},
            'Time Outside': {'None': 0, 'Little': 1, 'Moderate': 2, 'A lot': 3},
            'Screen Time': {'None': 0, 'Little': 1, 'Moderate': 2, 'A lot': 3},
        }
    },
    'Active': {
        'mode': 'map',
        'factors': {
            'Activity level': {'None': 0, 'Low': 1, 'Moderate': 2, 'High': 3},
        }
    },
    'Social': {
        'mode': 'map',
        'factors': {
            'Sociability level': {'None': 0, 'Little': 1, 'Moderate': 2, 'A lot': 3},
            'Social interaction quality': {'Poor': 1, 'OK': 2, 'Good': 3},
        }
    },
    'Weather': {
        'mode': 'map',
        'factors': {
            'Active level': {'None': 0, 'Low': 1, 'Moderate': 2, 'High': 3},
        }
    },
    'Symptom': {
        'mode': 'map',
        'factors': {}
    },
    'Sleep': {
        'mode': 'map',
        'factors': {}
    },
    'Nutrition': {
        'mode': 'map',
        'factors': {
            'Sugar': {'Little': 1, 'Moderate': 2, 'A Lot': 3},
            'Protein': {'Little': 1, 'Moderate': 2, 'A Lot': 3},
        }
    },
    'Mood': {
        'mode': 'mood',
        'factors': {}
    },
    'Med/Supplements': {
        'mode': 'preserve',
        'factors': {}
    },
    'Bowel Movements': {
        'mode': 'count_retain',
        'factors': {}
    },
    'Gratitudes': {
        'mode':'count_retain',
        'factors': {}
    }
}

In [5]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
data_dir = os.path.abspath(os.path.join(parent_dir, 'data'))
file_path = os.path.join(data_dir, 'bearable-export-28-12-2024.csv')


current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
data_dir = os.path.abspath(os.path.join(parent_dir, 'data'))
file_path = os.path.join(data_dir, 'bearable-export-28-12-2024.csv')

df=pd.read_csv(file_path)
df.head()

#PIPE functions

#remove rows that have category 'Health Measurement' 
df = df[df['category'] != 'Health measurements']

def convert_to_datetime(df, column_name, date_format=None):
    """
    Converts a column of dates in string format to pandas datetime format.
    Removes ordinal suffixes (e.g., '1st', '2nd') before conversion.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column containing dates.
        date_format (str): Optional date format for parsing (e.g., '%d %b %Y').

    Returns:
        pd.DataFrame: DataFrame with the converted datetime column.
    """
    try:
        # Step 1: Remove ordinal suffixes
        df[column_name] = df[column_name].str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)

        # Step 2: Convert to datetime
        df[column_name] = pd.to_datetime(df[column_name], format=date_format, errors='coerce')

        # Step 3: Log invalid rows
        invalid_dates = df[df[column_name].isna()]
        if not invalid_dates.empty:
            print("\n⚠️ WARNING: The following rows could not be converted to datetime:")
            print(invalid_dates)
    except Exception as e:
        print(f"\n❌ ERROR: Failed to convert column '{column_name}' to datetime.")
        print(e)
    
    return df

    #For rows with category 'Symptoms', group by dates/detail so we only have a symptom (currently the data keep 4 rows, redundant)
def clean_symptoms(df):
    symptom_rows =df['category']=='Symptom'

      # Clean 'detail' by removing parenthesis and level terms
    df.loc[symptom_rows, 'detail'] = df.loc[symptom_rows, 'detail'].str.replace(r"\s*\(.*\)", "", regex=True)

    grouped_symptom_rows=(
        df[symptom_rows]
        .groupby(['date','category','detail'], as_index=False)
        .first()
    )
    df=pd.concat([df[~symptom_rows], grouped_symptom_rows], ignore_index=True)
    return df

#For sleep, ditch rows if it has a time entry in 'Rating'.
def clean_sleep(df):
    df=df[~((df['category']=='Sleep') & (df['rating/amount'].notna()))]
    df=df[~((df['category']=='Sleep') & (df['detail']=='(Synced)'))]

    return df
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["  # Emoji ranges
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F680-\U0001F6FF"  # Transport & map symbols
        "\U0001F700-\U0001F77F"  # Alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric shapes extended
        "\U0001F800-\U0001F8FF"  # Supplemental arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
        "\U0001FA00-\U0001FA6F"  # Chess symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text).strip()  # Remove emojis and strip extra spaces

# Function to look into details, split by delimitter |, make new rows
def process_data(df):
    rows = []

    for _, row in df.iterrows():
        rating_amount = row['rating/amount']
        detail = row['detail']

        
        if pd.notna(rating_amount):
            # Retain rows with existing rating/amount
            rows.append(row.to_dict())
        elif pd.notna(detail):
            # Remove emojis and split by '|'
            clean_detail = remove_emojis(detail)
            factors = clean_detail.split(' | ')
            for factor in factors:
                rows.append({
                    'date': row['date'],
                    'category': row['category'],
                    'detail': factor.strip(),
                    'rating/amount': None  # Mark as new for processing
                })

    return pd.DataFrame(rows)

def categorize_data(df, rules):
    rows = []

    for _, row in df.iterrows():
        category = row['category']
        detail = row['detail']
        rating_amount = row['rating/amount']  # Preserve original value

        # Check the mode for the category
        if category in rules:   
            mode = rules[category].get('mode', 'binary')
            factors_rules = rules[category].get('factors', {})

            if mode == 'mood':  # Special case for Mood
                # Step 1: Create a row for Overall Mood
                if pd.notna(rating_amount):
                    rows.append({
                        'date': row['date'],
                        'category': category,  # New category
                        'detail': 'Overall Mood',  # Use the numerical rating as the detail
                        'rating/amount': rating_amount  # Fixed value of 1
                    })

            elif mode == 'map':
                # Handle cases like "Activity Level - Moderate"
                value = rating_amount if pd.notna(rating_amount) else 1 # Default binary value

                if '-' in detail:
                    name, level = map(str.strip, detail.split('-', 1))
                    if name in factors_rules and level in factors_rules[name]:
                        value = factors_rules[name][level]
                    detail = name  # Standardize the detail name
                rows.append({
                    'date': row['date'],
                    'category': category,
                    'detail': detail,
                    'rating/amount': value 
                })

            elif mode == 'count_retain':
                # Default binary mode
                rows.append({
                    'date': row['date'],
                    'category': category,
                    'detail': detail,
                    'rating/amount': 1
                })
        else:
            # For unmatched categories, preserve the original row
            rows.append(row.to_dict())

    # Convert processed rows back into a DataFrame
    processed_df = pd.DataFrame(rows)

    return processed_df

def pivot_data_multiindex(df):

    duplicate_rows = df[df.duplicated(subset=['category','date', 'detail','rating/amount'], keep=False)]
    if not duplicate_rows.empty:
        print("\n⚠️ WARNING: Duplicate rows detected before pivoting:")
        print(duplicate_rows)

    # Create the pivot table
    pivoted_df = df.pivot_table(
        index='date',                       # Group by date
        columns=['category', 'detail'],     # Multi-index columns with category and detail
        values='rating/amount',             # Values from 'rating/amount'
        aggfunc='sum',                      # Aggregate duplicates by sum
        fill_value=0                        # Fill missing values with 0
    )

    # Flatten the multi-index columns
    pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
    
    # Reset index to include 'date' as a column
    pivoted_df = pivoted_df.reset_index()
    
    return pivoted_df
df=(
    df
    .pipe(lambda x: convert_to_datetime(x, column_name='date'))
    .pipe(clean_symptoms)
    .pipe(clean_sleep)
    .pipe(process_data)  # Process rows to split and handle keywords
    .pipe(categorize_data, rules)  # Categorize rows based on rules
    .pipe(pivot_data_multiindex)
    
)
df.to_csv()




,date,date formatted,weekday,time of day,category,rating/amount,detail,notes
0,2024-06-28,2024-06-28,Friday,00:09,Mood,7,Optimistic | Hopeful | Great | Exhausted,NaN
1,2024-06-28,2024-06-28,Friday,pre,Symptom,2,Fatigue (Moderate),NaN
2,2024-06-28,2024-06-28,Friday,am,Symptom,2,Fatigue (Moderate),NaN
3,2024-06-28,2024-06-28,Friday,mid,Symptom,2,Fatigue (Moderate),NaN
4,2024-06-28,2024-06-28,Friday,pm,Symptom,2,Fatigue (Moderate),NaN
...,...,...,...,...,...,...,...,...
3410,2024-12-27,2024-12-27,Friday,all day,Weather,NaN,☁️ Cloudy,NaN
3411,2024-12-27,2024-12-27,Friday,all day,Creative,NaN,Piano,NaN
3415,2024-12-28,2024-12-28,Saturday,NaN,Sleep,NaN,NaN,(Synced)
3416,2024-12-28,2024-12-28,Saturday,NaN,Sleep quality,4,Good,NaN


In [105]:
df.head()

,date,Active_Activity level,Active_Basketball,Active_Climbing,Active_Cycling,Active_Pushups and Squats,Active_Table Tennis,Active_Walk,Active_Weights,Active_Yoga,...,Symptom_Indigestion,Symptom_Sinus congestion,Symptom_Stomachache,Symptom_Voice irritation,Weather_Cloudy,Weather_Cold,Weather_Pollen,Weather_Rain,Weather_Sunny,Weather_Windy
0,10th Aug 2024,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1,10th Dec 2024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,10th Jul 2024,2,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
3,10th Nov 2024,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,10th Oct 2024,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [113]:
df.dtypes

date              object
date formatted    object
weekday           object
time of day       object
category          object
rating/amount     object
detail            object
notes             object
dtype: object